# Financial Analysis

The main jupyter notebook for the analysis of Finnish election candidate finances. *This will, in all likelihood, not remain one file once exploration is complete*. Potential avenues include:
- General regression analysis, observing how much different factors at play (where a candidate got their money from, how they spent it, and other correlatory effects impacted the election results).
    - Previously held electoral experience, financial information
- Isolate anomalies, or people outside of the norm. I.e. Candidate 8 appears to have been very successful even though they spent very little, why is that?
- Who funds who, who spends where based on party, age
- Comparison on effectiveness for different types of ad spending
- Whether self funding is useful/successful

Important drawbacks to remember:
- This information only focuses on the winners of the election, rather then the losers. May be a winner bias, i.e. losers did spend more on self fundraising but those candidates who were already successful used it more effectively.
- Candidates in 2025 who are present in both the municipal and welfare district elections filed one financial report simultaneously. This was not the case before.
- Correlation != causuation and information can make things worse. Obvious but I'm writing it anyways because it must always be kept in mind.



In [18]:
# PREPARING DATA FOR MODEL AND VISUALIZATION
# STEP 1 - Importing Sheets
import pandas as pd
pd.set_option('display.max_columns', None)

kuntavaalit_ehdokas_sarakkeet = list(pd.read_csv("files/Results_title_rows_EN_ehdokas.csv")) # Saved as a list to apply to any candidate voting information, because header wasn't inculuded in the data from the Ministry of Justice.

years_data = {
    2012: {
        'finance': "files/finances/Liite_10_Kuntavaalit_2012_VI.csv",
        'results': "files/election_results_by_candidate/kv-2012_teat_maa.csv"
    },
    2017: {
        'finance': "files/finances/Liite_12_Kuntavaalit_2017_VI.csv",
        'results': "files/election_results_by_candidate/kv-2017_teat_maa.csv"
    },
    2021: {
        'finance': "files/finances/Liite_14_Kuntavaalit_2021_VI.csv",
        'results': "files/election_results_by_candidate/kv-2021_teat_maa.csv"
    },
    2025: {
        'finance': "files/finances/E_VI_kuntavaalit2025.csv",
        'results': "files/election_results_by_candidate/kv-2025_teat_maa.csv"
    }
}

def process_year_data(year, finance_path, results_path):
    """Process election data for a specific year."""

    kuntavaalit_rahoitus = pd.read_csv(finance_path, sep=";", quotechar="'", on_bad_lines='warn')
    kuntavaalit_ehdokas = pd.read_csv(results_path, sep=";", on_bad_lines='warn', encoding='latin-1', names=kuntavaalit_ehdokas_sarakkeet, index_col=False)

    kuntavaalit_ehdokas["LR_cand-id"] = (
        kuntavaalit_ehdokas["Candidate number"].astype(str).str.strip() + "-" +
        kuntavaalit_ehdokas["Name of a municipality/electoral district/voting area in Finnish"].str.strip() + "-" + kuntavaalit_ehdokas["Area type"]
    )
    kuntavaalit_rahoitus["LR_cand-id"] = (
        kuntavaalit_rahoitus["Ehdokasnumero"].astype(str).str.strip() + "-" +
        kuntavaalit_rahoitus["Vaalipiiri/Kunta"].str.strip() + "-K"
    ) # We need to merge the candidates, but the IDs are per municipality. I create a new ID to ensure uniqueness across the whole data. UPDATE Oct 31st - realized that candidates were being duplicated because the ID matched with both the municipality and more specific voting district. Adding area type (K) to specify that it's searching for the municipality only.

    kuntavaalit_yhteensä = kuntavaalit_rahoitus.merge(kuntavaalit_ehdokas, how="left", on="LR_cand-id") # Merged on the left since a lot of duplicate data in the Ministry of Justice and I only want the candidates who actually have fundraising data of some kind (i.e. candidates in more then one municipality with no votes? My look suggests that they are only given the votes where they actually ran, and assumedly the financial data with self select, but INVESTIGATE MORE).

    kuntavaalit_yhteensä['year'] = year # Year column for classification later

    kuntavaalit_yhteensä['LR_cand-id'] = (kuntavaalit_yhteensä['LR_cand-id'] + "-" + str(year)) # For uniqueness when combining the entire dataset.

    return kuntavaalit_yhteensä

all_years_data = []
for year, paths in years_data.items():
    print(f"Processing {year}...")
    year_df = process_year_data(year, paths['finance'], paths['results'])
    all_years_data.append(year_df)

kuntavaalit_yhteensä_all = pd.concat(all_years_data, ignore_index=True)
print(f"In total, there were {len(all_years_data[0])} candidates recorded from 2012; {len(all_years_data[1])} from 2017; {len(all_years_data[2])} from 2021; and {len(all_years_data[3])} from 2025.")

Processing 2012...


C:\Users\kaife\AppData\Local\Temp\ipykernel_24104\4235754406.py:31: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  kuntavaalit_ehdokas = pd.read_csv(results_path, sep=";", on_bad_lines='warn', encoding='latin-1', names=kuntavaalit_ehdokas_sarakkeet, index_col=False)


Processing 2017...


C:\Users\kaife\AppData\Local\Temp\ipykernel_24104\4235754406.py:30: DtypeWarning: Columns (31,34) have mixed types. Specify dtype option on import or set low_memory=False.
  kuntavaalit_rahoitus = pd.read_csv(finance_path, sep=";", quotechar="'", on_bad_lines='warn')
C:\Users\kaife\AppData\Local\Temp\ipykernel_24104\4235754406.py:31: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  kuntavaalit_ehdokas = pd.read_csv(results_path, sep=";", on_bad_lines='warn', encoding='latin-1', names=kuntavaalit_ehdokas_sarakkeet, index_col=False)


Processing 2021...


C:\Users\kaife\AppData\Local\Temp\ipykernel_24104\4235754406.py:30: DtypeWarning: Columns (34,40) have mixed types. Specify dtype option on import or set low_memory=False.
  kuntavaalit_rahoitus = pd.read_csv(finance_path, sep=";", quotechar="'", on_bad_lines='warn')
C:\Users\kaife\AppData\Local\Temp\ipykernel_24104\4235754406.py:31: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  kuntavaalit_ehdokas = pd.read_csv(results_path, sep=";", on_bad_lines='warn', encoding='latin-1', names=kuntavaalit_ehdokas_sarakkeet, index_col=False)


Processing 2025...


C:\Users\kaife\AppData\Local\Temp\ipykernel_24104\4235754406.py:30: DtypeWarning: Columns (8,11,12,16,17,18,19,20,21,22,23,24,25,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  kuntavaalit_rahoitus = pd.read_csv(finance_path, sep=";", quotechar="'", on_bad_lines='warn')
C:\Users\kaife\AppData\Local\Temp\ipykernel_24104\4235754406.py:31: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  kuntavaalit_ehdokas = pd.read_csv(results_path, sep=";", on_bad_lines='warn', encoding='latin-1', names=kuntavaalit_ehdokas_sarakkeet, index_col=False)


In total, there were 19069 candidates recorded from 2012; 17532 from 2017; 17395 from 2021; and 16388 from 2025.


In [55]:
import copy
import numpy as np

# STEP 2 - Creating joint analysis sheet

municipal_macro_results = pd.read_csv("files/outputs/municipal_macro_per_year.csv") # Getting municipal macro results for analysis purposes.

## Creating an incumbency single column value
def determine_incumbency(row):
    """Determine the highest level of incumbency for a candidate."""
    if row["Member of Parliament"] == "1":
        return "European Parliament"
    elif row["Member of the European Parliament"] == "1":
        return "Parliament"
    elif row["Municipal councillor"] == "1":
        return "Municipal"
    elif row["County councillor"] == "1":
        return "County"
    else:
        return "None"

def clean_numeric_columns(df, columns):
    """Convert columns with comma decimals to numeric type."""
    for col in columns: # NOTE: Currently will crash if it runs into a non string in a column
        df[col] = df[col].str.replace(" ", "").str.replace(",", ".")
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

def set_minor_values(df, column):
    df[column]


kuntavaalit_yhteensä_all_t = copy.deepcopy(kuntavaalit_yhteensä_all)

kuntavaalit_yhteensä_all_t["LR_incumbency"] = kuntavaalit_yhteensä_all_t.apply(
    determine_incumbency, axis=1
)

kuntavaalit_yhteensä_all_t = kuntavaalit_yhteensä_all_t.rename(columns={
    'Percentage (%) of the total number of votes': 'vote_prct',
    'Ehdolla molemmissa samanaikaisissa vaaleissa': 'multi_election',
    'Vaalikampanjan kulut yhteensa': 'total_expenses',
    "Vaalikampanjan rahoitus yhteensa": "total_raised",
    "Vaalipiiri/Kunta": "municipality",
    "Age on election day": "age",
    "Puolue": "party",
    "2.1 Rahoitus sisaltaa omia varoja yhteensa": "tf_self_funding",
    "2.2 Rahoitus sisaltaa ehdokkaan ja tukiryhman ottamia lainoja yhteensa": "tf_loans",
    "2.3 Rahoitus sisaltaa yksityishenkiloilta saatua tukea yhteensa": "tf_private_individuals",
    "2.4 Rahoitus sisaltaa yrityksilta saatua tukea yhteensa": "tf_companies",
    "2.5 Rahoitus sisaltaa puolueelta saatua tukea yhteensa": "tf_party",
    "2.6 Rahoitus sisaltaa puolueyhdistyksilta saatua tukea yhteensa": "tf_party_associations",
    "2.7 Rahoitus sisaltaa muilta tahoilta saatua tukea yhteensa": "tf_others",
    "Sanoma- ilmaisjakelu- ja aikakauslehdet": "sl_newspapers_periodicals",
    "Radio": "sl_radio",
    "Televisio": "sl_television",
    "Tietoverkot": "sl_information_networks",
    "Muut viestintavalineet": "sl_other_media",
    "Ulkomainonta": "sl_outdoor_advertising",
    "Vaalilehtien esitteiden ja muun painetun materiaalin hankinta": "sl_printed_material_procurement",
    "Mainonnan suunnittelu": "sl_advertising_planning",
    "Vaalitilaisuudet": "sl_rallies",
    "Vastikeellisen tuen hankintakulut": "sl_fundraising",
    "Muut kulut": "sl_other"
}) # Renaming for model calc. Doesn't like spaces/special characters

main_parties = ["SDP", "KOK", "VAS", "VIHR", "KD", "KESK", "KOK", "PS"]
kuntavaalit_yhteensä_all_t['party'] = np.where(
    kuntavaalit_yhteensä_all_t['party'].isin(main_parties),
    kuntavaalit_yhteensä_all_t['party'],
    'Other'
) # We want to make sure that the small parties are not blurring the information or making sorting difficult.

kuntavaalit_yhteensä_all_t = clean_numeric_columns(kuntavaalit_yhteensä_all_t, ["total_raised", "total_expenses", "tf_self_funding", "tf_loans", "tf_private_individuals", "tf_companies", "tf_party", "tf_party_associations", "tf_others", "sl_newspapers_periodicals", "sl_radio", "sl_television", "sl_information_networks", "sl_other_media", "sl_outdoor_advertising", "sl_printed_material_procurement", "sl_advertising_planning", "sl_rallies", "sl_fundraising", "sl_other"])

kuntavaalit_yhteensä_all_t["own_fund_prct"] = kuntavaalit_yhteensä_all_t["tf_self_funding"]/kuntavaalit_yhteensä_all_t["total_raised"] # Testing out in the regression analysis whether a % of own funding for campaigns is statistically relevant
kuntavaalit_yhteensä_all_t["full_name"] = kuntavaalit_yhteensä_all_t["Etunimet"] + " " + kuntavaalit_yhteensä_all_t["Sukunimi"] # I want to be able to easily call the full names of people
kuntavaalit_yhteensä_all_t["total_raised_per_1k"] = kuntavaalit_yhteensä_all_t["total_raised"]/1000 # Better way to see the effects on the regression analysis
kuntavaalit_yhteensä_all_t["vote_prct"] = kuntavaalit_yhteensä_all_t["vote_prct"]/10 # Vote percent is raised by a factor of 10
kuntavaalit_yhteensä_all_t["multi_election"] = kuntavaalit_yhteensä_all_t["multi_election"].fillna("Ei") # 2025 election had candidates file financial reporting when running in two elections. I want to isolate this effect from the acutal effect of money on one election.

# Municipality NUM candidates matching
lookup = municipal_macro_results.set_index(
    ['Name of a municipality/electoral district/voting area in Finnish', 'Year']
)['num_candidates'].to_dict()

# print(lookup)

kuntavaalit_yhteensä_all_t['num_candidates_in_municipality'] = kuntavaalit_yhteensä_all_t.apply(
    lambda row: lookup.get((row['municipality'], row['year']), None), axis=1
)

kuntavaalit_yhteensä_all_t["normalized_vote_prct"] = kuntavaalit_yhteensä_all_t['vote_prct'] * kuntavaalit_yhteensä_all_t['num_candidates_in_municipality'] / max(kuntavaalit_yhteensä_all_t['num_candidates_in_municipality']) # For display purposes. Candidates in smaller districts will have their "ability" overrepresented.

# Creating dummies
dummies_incumbency = pd.get_dummies(kuntavaalit_yhteensä_all_t['LR_incumbency'], prefix='incumbency')
dummies_multi = pd.get_dummies(kuntavaalit_yhteensä_all_t['multi_election'], prefix='multi_election')

kuntavaalit_yhteensä_all_t = pd.concat([kuntavaalit_yhteensä_all_t, dummies_incumbency, dummies_multi], axis=1)

print(len(kuntavaalit_yhteensä_all_t))
kuntavaalit_yhteensä_all_t = kuntavaalit_yhteensä_all_t.dropna(subset=['total_raised'])
kuntavaalit_yhteensä_all_t = kuntavaalit_yhteensä_all_t.drop(kuntavaalit_yhteensä_all_t[np.isinf(kuntavaalit_yhteensä_all_t['own_fund_prct'])].index)
kuntavaalit_yhteensä_all_t_helsinki = kuntavaalit_yhteensä_all_t.drop(kuntavaalit_yhteensä_all_t[kuntavaalit_yhteensä_all_t['municipality']!="Helsinki"].index)
kuntavaalit_yhteensä_all_t_helsinki_2025 = kuntavaalit_yhteensä_all_t_helsinki.drop(kuntavaalit_yhteensä_all_t_helsinki[kuntavaalit_yhteensä_all_t_helsinki['year']!=2025].index)
kuntavaalit_yhteensä_all_t_2025 = kuntavaalit_yhteensä_all_t.drop(kuntavaalit_yhteensä_all_t[kuntavaalit_yhteensä_all_t['year']!=2025].index)
print(len(kuntavaalit_yhteensä_all_t))

# print(municipal_macro_results["total_votes"].describe())
print(kuntavaalit_yhteensä_all_t["Total number of votes"].describe())

display(municipal_macro_results)
display(kuntavaalit_yhteensä_all_t)
# # means = kuntavaalit_yhteensä_2025.groupby('LR_incumbency')['total_expenses'].mean()
# print(means)
# print(kuntavaalit_yhteensä["Total number of votes"].sum())
# print(kuntavaalit_yhteensä_all_t[kuntavaalit_yhteensä_all_t["LR_cand-id"]=="216-Helsinki"])



70384
19565
count    19565.000000
mean       228.455047
std        517.960815
min          0.000000
25%         54.000000
50%        111.000000
75%        227.000000
max      29745.000000
Name: Total number of votes, dtype: float64


,Unnamed: 0,Name of a municipality/electoral district/voting area in Finnish,Year,num_candidates,total_votes
0,0,Akaa,2012,157,7590
1,1,Akaa,2017,134,7532
2,2,Akaa,2021,164,6867
3,3,Akaa,2025,132,6950
4,4,Alajärvi,2012,112,5384
...,...,...,...,...,...
1179,1179,Ähtäri,2025,75,2341
1180,1180,Äänekoski,2012,144,9201
1181,1181,Äänekoski,2017,126,8647
1182,1182,Äänekoski,2021,124,7650


,Ehdokasnumero,Kuntanumero,Etunimet,Sukunimi,Arvo/ammatti/tehtava,municipality,party,Ehdokkaan mahdollisen tukiryhman nimi,Vaalirahoitukseni ei ole ylittanyt 800 euroa,total_expenses,sl_newspapers_periodicals,sl_radio,sl_television,sl_information_networks,sl_other_media,sl_outdoor_advertising,sl_printed_material_procurement,sl_advertising_planning,sl_rallies,sl_fundraising,sl_other,total_raised,tf_self_funding,tf_loans,tf_private_individuals,"2.3 Ei sisalla yhtaan vahintaan 800 euron tukisuoritusta yksityishenkilolta (""x"")",2.3 Rahoitus sisaltaa yksityishenkiloilta saatua tukea yhteensa lisatietokentan teksti,tf_companies,"2.4 Ei sisalla yhtaan vahintaan 800 euron tukisuoritusta yrityksilta (""x"")",2.4 Rahoitus sisaltaa yrityksilta saatua tukea yhteensa lisatietokentan teksti,tf_party,"2.5 Ei sisalla yhtaan vahintaan 800 euron tukisuoritusta puolueelta (""x"")",2.5 Rahoitus sisaltaa puolueelta saatua tukea yhteensa lisatietokentan teksti,tf_party_associations,"2.6 Ei sisalla yhtaan vahintaan 800 euron tukisuoritusta puolueyhdistyksilta (""x"")",2.6 Rahoitus sisaltaa puolueyhdistyksilta saatua tukea yhteensa lisatietokentan teksti,tf_others,"2.7 Ei sisalla yhtaan vahintaan 800 euron tukisuoritusta muilta tahoilta (""x"")",2.7 Rahoitus sisaltaa muilta tahoilta saatua tukea yhteensa lisatietokentan teksti,2.8 Rahoitus sisaltaa valitettya tukea yhteensa,E. vapaaehtoinen erittely,LR_cand-id,Election type,Electoral district / county number,Municipality number,Area type,Voting area identifier,Abbreviation for an electoral district / a county in Finnish,Abbreviation for an electoral district / a county in Swedish,Permanent party identifier,Standard party number,List order number,Electoral alliance number,Abbreviated name of a political party/group in Finnish,Abbreviated name of a political party/group in Swedish,Abbreviated name of a political party/group in English,Candidate number,Name of a municipality/electoral district/voting area in Finnish,Name of a municipality/electoral district/voting area in Swedish,A person’s first name,A person’s last name,Gender,age,Profession,Code for place of residence,The name of a place of residence in Finnish,The name of a place of residence in Swedish,Candidate’s language,Member of the European Parliament,Member of Parliament,Municipal councillor,County councillor,Abbreviated name of the first comparison election,Number of votes in the first comparison election,Number of votes cast in advance,Number of votes cast on election day,Total number of votes,Percentage (%) of votes cast in advance,Percentage (%) of votes on election day,vote_prct,Elected information,Comparative index,Position,Final position,Calculation status,Calculation phase,Latest update,year,Saapumispaiva,Muokkauspaiva,multi_election,Aluevaalit 2025,Kuntavaalit 2025,LR_incumbency,own_fund_prct,full_name,total_raised_per_1k,num_candidates_in_municipality,normalized_vote_prct,incumbency_County,incumbency_European Parliament,incumbency_Municipal,incumbency_None,incumbency_Parliament,multi_election_Ei,multi_election_Kyllä
0,254,186,Tomi Antero,Riihimäki,"kauppat.kand., toimitusjohtaja",Järvenpää,KOK,NaN,NaN,2030.50,1770.00,NaN,NaN,NaN,NaN,184.50,76.00,NaN,NaN,NaN,NaN,2030.50,1846.00,NaN,NaN,X,NaN,184.5,X,NaN,NaN,X,NaN,NaN,X,NaN,NaN,X,NaN,NaN,NaN,254-Järvenpää-K-2012,K,2,186,K,****,UUS,NYL,4,1,10,9,KOK,SAML,NCP,254,Järvenpää,Träskända,Tomi ...,Riihimäki ...,1,54,"kauppat.kand., toimitusjohtaja ...",186,Järvenpää,Träskända,,,,1,,K-2008,0000000,41,63,104,7,6,0.6,1,393100,39,39,V,T,20121108140421,2012,NaN,NaN,Ei,NaN,NaN,Municipal,0.909136,Tomi Antero Riihimäki,2.03050,292,0.150645,False,False,True,False,False,True,False
8,90,186,Jani,Uhlenius,"muusikko, vapaa kirjoittaja",Järvenpää,SDP,NaN,NaN,1730.00,1346.00,NaN,NaN,NaN,NaN,NaN,384.00,NaN,NaN,NaN,NaN,1730.00,805.00,NaN,600.0,X,NaN,200.0,X,NaN,NaN,X,NaN,NaN,X,NaN,125.0,X,NaN,NaN,NaN,90-Järvenpää-K-2012,K,2,186,K,****,UUS,NYL,1,2,2,2,SDP,SDP,SDP,90,Järvenpää,Träskända,Jani ...,Uhlenius ...,1

In [31]:
from sklearn import datasets, linear_model
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt



# print(len(kuntavaalit_yhteensä_all_t_helsinki_2025))
# print(len(kuntavaalit_yhteensä_all_t_helsinki))
# display(kuntavaalit_yhteensä_all_t_helsinki_2025)
# display(kuntavaalit_yhteensä_all_t_helsinki)


# model_2025 = smf.ols('vote_prct ~ total_raised + C(LR_incumbency, Treatment(reference="None")) + C(multi_election)',
#                 data=kuntavaalit_yhteensä_all_t_helsinki_2025).fit()
# print(f"Observations used in model: {model_2025.nobs}")
# print(model_2025.summary())

model_all = smf.ols('vote_prct ~ total_raised + C(LR_incumbency, Treatment(reference="None")) + C(multi_election)',
                data=kuntavaalit_yhteensä_all_t_helsinki).fit()
print(f"Observations used in model: {model_all.nobs}")
print(model_all.summary())

# model_all = smf.ols('vote_prct ~ "tf_self_funding" + "tf_loans" + "tf_private_individuals" + "tf_companies" + "tf_party" + "tf_party_associations" + "tf_others" + C(LR_incumbency, Treatment(reference="None")) + C(multi_election)',
#                 data=kuntavaalit_yhteensä_all_t_helsinki).fit()
# print(f"Observations used in model: {model_all.nobs}")
# print(model_all.summary())



# # model = smf.ols('vote_prct ~ total_raised + C(LR_incumbency) + C(multi_election) + C(Puolue)',
# #                 data=kuntavaalit_yhteensä_2025_helsinki).fit()
#

Observations used in model: 602.0
                            OLS Regression Results                            
Dep. Variable:              vote_prct   R-squared:                       0.468
Model:                            OLS   Adj. R-squared:                  0.465
Method:                 Least Squares   F-statistic:                     131.4
Date:                Fri, 31 Oct 2025   Prob (F-statistic):           1.96e-80
Time:                        18:13:09   Log-Likelihood:                -1763.3
No. Observations:                 602   AIC:                             3537.
Df Residuals:                     597   BIC:                             3559.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                                                           coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------

In [32]:
import statsmodels.formula.api as smf
# model_2025 = smf.ols('vote_prct ~ total_raised + C(LR_incumbency, Treatment(reference="None")) + C(multi_election)',
#                 data=kuntavaalit_yhteensä_all_t_2025).fit()
# print(f"Observations used in model: {model_2025.nobs}")
# print(model_2025.summary())
#tf_self_funding + tf_loans + tf_private_individuals + tf_companies + tf_party + tf_party_associations + tf_others

model_all = smf.ols('vote_prct ~ total_raised + num_candidates_in_municipality + C(LR_incumbency, Treatment(reference="None")) + C(multi_election)',
                data=kuntavaalit_yhteensä_all_t).fit()
print(f"Observations used in model: {model_all.nobs}")
print(model_all.summary())


Observations used in model: 19565.0
                            OLS Regression Results                            
Dep. Variable:              vote_prct   R-squared:                       0.291
Model:                            OLS   Adj. R-squared:                  0.291
Method:                 Least Squares   F-statistic:                     1146.
Date:                Fri, 31 Oct 2025   Prob (F-statistic):               0.00
Time:                        18:13:21   Log-Likelihood:                -79771.
No. Observations:               19565   AIC:                         1.596e+05
Df Residuals:                   19557   BIC:                         1.596e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                                                           coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------

In [44]:
kuntavaalit_yhteensä_all_t.to_csv("files/outputs/financial_analysis_data.csv")